# House Price - Data Processing with TF.Transform

This notebook will report all the required code to create and export a data processing pipeline using Apache Beam and TF.Transform.

Whenever possible the notebook will be coded to support cloud run on GCP and local run.

## Set global parameters

In [1]:
PROJECT = 'gcp-playground' # change to your project_Id
BUCKET = 'gcs-cloudml'     # change to your bucket name
REGION = 'region'          # change to your region


ROOT_DIR = 'data'          # directory where the data are located is stored locally or on GCS
INPUT_FILE = "train.csv"
OUTPUT_DIR = "data/output"

## Import required modules

In [2]:
import os
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['ROOT_DIR'] = ROOT_DIR
os.environ['RUN_LOCAL'] = str(RUN_LOCAL)

In [3]:
# Import data processing libraries
import pandas as pd
import tensorflow as tf
# Import Tensorflow Transform
import tensorflow_transform as tft
from tensorflow_transform.tf_metadata import dataset_metadata
from tensorflow_transform.tf_metadata import dataset_schema
# Import Apache Beam
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
# Python shutil module enables us to operate with file objects easily and without diving into file objects a lot.
import shutil
# Show the currently installed version of TensorFlow
print("Tensorflow version: {}".format(tf.__version__))

Tensorflow version: 2.3.2


## Input source: CSV Local

In [4]:
# Import data locally only the fist 100 rows to support
# the creation of the processing functions

# TODO update path to bucket
path_to_csv = os.path.join(ROOT_DIR, INPUT_FILE)
data = pd.read_csv(path_to_csv, nrows=100)

In [13]:
data.columns

Index(['POSTED_BY', 'UNDER_CONSTRUCTION', 'RERA', 'BHK_NO.', 'BHK_OR_RK',
       'SQUARE_FT', 'READY_TO_MOVE', 'RESALE', 'ADDRESS', 'LONGITUDE',
       'LATITUDE', 'TARGET(PRICE_IN_LACS)'],
      dtype='object')

In [5]:
data.head()

,POSTED_BY,UNDER_CONSTRUCTION,RERA,BHK_NO.,BHK_OR_RK,SQUARE_FT,READY_TO_MOVE,RESALE,ADDRESS,LONGITUDE,LATITUDE,TARGET(PRICE_IN_LACS)
0,Owner,0,0,2,BHK,1300.236407,1,1,"Ksfc Layout,Bangalore",12.969910,77.597960,55.0
1,Dealer,0,0,2,BHK,1275.000000,1,1,"Vishweshwara Nagar,Mysore",12.274538,76.644605,51.0
2,Owner,0,0,2,BHK,933.159722,1,1,"Jigani,Bangalore",12.778033,77.632191,43.0
3,Owner,0,1,2,BHK,929.921143,1,1,"Sector-1 Vaishali,Ghaziabad",28.642300,77.344500,62.5
4,Dealer,1,0,2,BHK,999.009247,0,1,"New Town,Kolkata",22.592200,88.484911,60.5


In [6]:
# Convert csv in a bigquery like processor
data_bq_like = list(data.iloc[:5].to_dict("index").values()) # sample just the first 10 rows to test the processing functions

## Create raw data metadata

In [7]:
CATEGORICAL_FEATURE_NAMES = ['POSTED_BY', 'UNDER_CONSTRUCTION', 'RERA', 
                             'BHK_OR_RK', 'READY_TO_MOVE', 'RESALE', 
                             'ADDRESS']
#CATEGORICAL_FEATURE_MAX_VALUES = [24]

NUMERIC_FEATURE_NAMES = ['BHK_NO.',  'SQUARE_FT', 'LONGITUDE','LATITUDE']
#BUCKET_FEATURE_BUCKET_COUNT = [100]

TARGET_FEATURE_NAME = "TARGET(PRICE_IN_LACS)"

In [8]:
def create_raw_metadata():
    
    raw_data_schema = {}
    
 
    # target feature schema
    raw_data_schema[TARGET_FEATURE_NAME]= dataset_schema.ColumnSchema(
            tf.float32, [], dataset_schema.FixedColumnRepresentation())
    
    # categorical features schema
    raw_data_schema.update({column_name : dataset_schema.ColumnSchema(
        tf.string, [], dataset_schema.FixedColumnRepresentation())
                            for column_name in CATEGORICAL_FEATURE_NAMES})
    
    # numerical features schema
    raw_data_schema.update({ column_name : dataset_schema.ColumnSchema(
        tf.float32, [], dataset_schema.FixedColumnRepresentation())
                            for column_name in NUMERIC_FEATURE_NAMES})
    
    # create dataset_metadata given raw_schema
    raw_data_metadata = dataset_metadata.DatasetMetadata(
        dataset_schema.Schema(raw_data_schema))
    
    return raw_data_metadata

## Input source: BigQuery

In [ ]:
from google.cloud import bigquery

def create_query(phase, hash_column, table_name, EVERY_N):
    """Creates a query with the proper splits.
    Args:
        phase: str, train, valid.
        hash_column: str, is the column that we will use to perform the split
        EVERY_N: int, take an example EVERY_N rows.

    Returns:
        Query string with the proper splits.
    """
    base_query = """
    SELECT * FROM {0}
    """.format(table_name)
    
    if EVERY_N is None:
        if phase=="train":
            # training
            query = """{0} AND ABS(MOD(FARM_FINGERPRINT(CAST
            ({1} AS STRING), 10)) < 8""".format(base_query, hash_column)
        else:
            query = """{0} AND ABS(MOD(FARM_FINGERPRINT(CAST(
            {1} AS STRING), 10)) > 8""".format(base_query, hash_column)
    else:
        query = """{0} AND ABS(MOD(FARM_FINGERPRINT(CAST(
        {1} AS STRING)), {2})) = 2""".format(base_query,hash_column, EVERY_N)

    return query

In [10]:
def is_valid(inputs):
    """Check to make sure the inputs are valid.
    Args:
        inputs: dict, dictionary of TableRow data from BigQuery.

    Returns:
        True if the inputs are valid and False if they are not.
    """
    try:
        underconstruction = inputs["UNDER_CONSTRUCTION"]
        return underconstruction >= 0 and underconstruction  <2
    except:
        return False

In [27]:
def preprocess_tft(inputs):
    """
    Preprocess the features and add engineered features with tf transform.

    Args:
        dict, dictionary of TableRow data from BigQuery.

    Returns:
        Dictionary of preprocessed data after scaling and feature engineering.
    """
    import datetime
    print(inputs)
    # create a new dictionary that will contain only those relevan features selected
    result = {}
    
    # Report the label in the processed data set
    result["target_price_lacs"] = tf.identity(inputs["TARGET(PRICE_IN_LACS)"])
    
    # Build a vocobulary
    result["address"] = tft.compute_and_apply_vocabulary(inputs["ADDRESS"].split(",")[-1]) #remove the address and leave the city
    result["posted_by"] = tf.compute_and_apply_vocabulary(inputs["POSTED_BY"])
    result["bhk_or_rk"] = tf.compute_and_apply_vocabulary(inputs["BHK_OR_RK"])
    
    # Keep the  categorical features untouch
    result["under_construction"] = tf.identity(inputs["UNDER_CONSTRUCTION"])
    result["rera"] = tf.identity(inputs["RERA"])
    result["ready_to_move"] = tf.identity(inputs["READY_TO_MOVE"])
    result["resale"] = tf.identity(inputs["RESALE"])

    # Scaling numeric values
    result['BHK_NO'] = (tft.scale_to_0_1(inputs['BHK_NO']))
    result['SQUARE_FT'] = (tft.scale_to_0_1(inputs['SQUARE_FT']))

    # Bucketize
    result['longitude_bucket'] = tft.bucketize(inputs["LONGITUDE"], num_buckets=100)
    result['latitude_bucket'] = tft.bucketize(inputs["LATITUDE"], num_buckets=100)
    
    
    # Engineered features
    # basic implementation of a feature created to (potentially) improve the prediction
    result["avg_room_sq.ft"] = tft.scale_to_0_1(inputs['SQUARE_FT'] / inputs['BHK_NO'])
    
    return result

## Process function Apache Beam

In [ ]:
def preprocess(in_test_mode):
    """Sets up preprocess pipeline.

    Args:
        in_test_mode: bool, False to launch DataFlow job, True to run locally.
    """
    import os
    import os.path
    import tempfile
    from apache_beam.io import tfrecordio
    from tensorflow_transform.coders import example_proto_coder
    from tensorflow_transform.tf_metadata import dataset_metadata
    from tensorflow_transform.tf_metadata import dataset_schema
    from tensorflow_transform.beam import tft_beam_io
    from tensorflow_transform.beam.tft_beam_io import transform_fn_io

    job_name = 'house_price_tftprocessing' + '-'
    job_name += datetime.datetime.now().strftime('%y%m%d-%H%M%S')
    if in_test_mode:
        import shutil
        print('Launching local job ...')
        OUTPUT_DIR = './procesing_tft'
        shutil.rmtree(OUTPUT_DIR, ignore_errors=True)
        EVERY_N = 100
    else:
        print('Launching Dataflow job {} ...'.format(job_name))
        OUTPUT_DIR = 'gs://{0}/houseprice/processing_tft/'.format(BUCKET)
        import subprocess
        subprocess.call('gsutil rm -r {}'.format(OUTPUT_DIR).split())
        EVERY_N = None

    options = {
        'staging_location': os.path.join(OUTPUT_DIR, 'tmp', 'staging'),
        'temp_location': os.path.join(OUTPUT_DIR, 'tmp'),
        'job_name': job_name,
        'project': PROJECT,
        'num_workers': 1,
        'max_num_workers': 1,
        'teardown_policy': 'TEARDOWN_ALWAYS',
        'no_save_main_session': True,
        'direct_num_workers': 1,
        'extra_packages': ['tensorflow_transform-0.24.0-py3-none-any.whl']
        }

    opts = beam.pipeline.PipelineOptions(flags=[], **options)
    if in_test_mode:
        RUNNER = 'DirectRunner'
    else:
        RUNNER = 'DataflowRunner'

    # Set up raw data metadata
    raw_data_schema = create_raw_metadata()

    # Run Beam
    with beam.Pipeline(RUNNER, options=opts) as p:
        with beam_impl.Context(temp_dir=os.path.join(OUTPUT_DIR, 'tmp')):
            # Save the raw data metadata
            (raw_data_metadata |
                'WriteInputMetadata' >> tft_beam_io.WriteMetadata(
                    os.path.join(
                        OUTPUT_DIR, 'metadata/rawdata_metadata'), pipeline=p))

            # Read training data from bigquery and filter rows
            raw_data = (p 
                        | 'Read train set' >> beam.io.Read(beam.io.BigQuerySource(query=create_query("train", "ADDRESS", <table_name>),
                                                                              use_standard_sql=True)) 
                        | 'Validate data' >> beam.Filter(is_valid))

            raw_dataset = (raw_data, raw_data_metadata)

            # Analyze and transform training data
            transformed_dataset, transform_fn = (raw_dataset 
                                                 | beam_impl.AnalyzeAndTransformDataset(preprocess_tft))
            
            transformed_data, transformed_metadata = transformed_dataset

            # Save transformed train data to disk in efficient tfrecord format
            transformed_data | 'WriteTrainData' >> tfrecordio.WriteToTFRecord(os.path.join(OUTPUT_DIR, 'train'),
                                                                              file_name_suffix='.gz',
                                                                              coder=example_proto_coder.ExampleProtoCoder(transformed_metadata.schema))

            
            # Read eval data from bigquery and filter rows
            raw_test_data = (p 
                             | 'Read eval set' >> beam.io.Read(beam.io.BigQuerySource(query=create_query("valid", "ADDRESS", <table_name>),
                                                                                        use_standard_sql=True)) 
                             | 'eval_filter' >> beam.Filter(is_valid))

            raw_test_dataset = (raw_test_data, raw_data_metadata)

            # Transform eval data
            transformed_test_dataset = ((raw_test_dataset, transform_fn) | beam_impl.TransformDataset())
            transformed_test_data, _ = transformed_test_dataset

            # Save transformed train data to disk in efficient tfrecord format
            (transformed_test_data | 'WriteTestData' >> tfrecordio.WriteToTFRecord(os.path.join(OUTPUT_DIR, 'eval'), 
                                                                                   file_name_suffix='.gz',
                                                                                   coder=example_proto_coder.ExampleProtoCoder(transformed_metadata.schema)))

            # Save transformation function to disk for use at serving time
            (transform_fn | 'WriteTransformFn' >> transform_fn_io.WriteTransformFn(os.path.join(OUTPUT_DIR, 'metadata')))

In [ ]:
# Change to False to run on DataFlow
preprocess(in_test_mode=True)